# Advanced Example: Random Environment Closed Queueing Network

This example demonstrates a closed queueing network that can be used in random environment models:
- **Nodes**: Delay node + PS Queue
- **Class**: Single closed class with N jobs
- **Service**: Parameterized exponential service rates

The model creates a simple 2-station closed network with circular routing.

In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
def renv_genqn(rate, N):
    """
    Create a simple closed queueing network for random environment models.
    
    Args:
        rate: List of service rates [rate1, rate2] for the two stations
        N: Population size (number of jobs in the closed network)
    
    Returns:
        Network: A closed queueing network model
    """
    model = Network('qn1')
    
    # Block 1: nodes
    delay = Delay(model, 'Queue1')
    queue = Queue(model, 'Queue2', SchedStrategy.PS)
    
    # Block 2: classes
    jobclass = ClosedClass(model, 'Class1', N, delay, 0)
    
    # Set service rates
    delay.set_service(jobclass, Exp(rate[0]))
    queue.set_service(jobclass, Exp(rate[1]))
    
    # Block 3: topology (circular routing)
    P = model.init_routing_matrix()
    P.add_route(jobclass, delay, queue, 1.0)
    P.add_route(jobclass, queue, delay, 1.0)
    model.link(P)
    
    return model

# Create the model with example parameters
model = renv_genqn(rate=[2.0, 3.0], N=5)

## About Random Environment Models

Random environment queueing models feature:
- **Base network**: A queueing network (like this closed QN)
- **Environment process**: A Markov chain that modulates network parameters
- **State-dependent rates**: Service/arrival rates depend on environment state

This simple closed network is a building block for:
- Markov-modulated queueing systems
- Systems with random breakdowns
- Networks with time-varying workloads

In [3]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)


=== Solver Results ===
Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
 Queue1   Class1 1.4787 1.4787 0.5000  0.5000 2.9575 2.9575
 Queue2   Class1 3.5213 0.9858 1.1906  1.1906 2.9575 2.9575

MVA Solver:
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Queue1   Class1  1.4787  1.4787  0.5000  0.5000  2.9575  2.9575
1  Queue2   Class1  3.5213  0.9858  1.1906  1.1906  2.9575  2.9575
Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
 Queue1   Class1 1.4787 1.4787 0.5000  0.5000 2.9575 2.9575
 Queue2   Class1 3.5213 0.9858 1.1906  1.1906 2.9575 2.9575

CTMC Solver:
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Queue1   Class1  1.4787  1.4787  0.5000  0.5000  2.9575  2.9575
1  Queue2   Class1  3.5213  0.9858  1.1906  1.1906  2.9575  2.9575
Station JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
 Queue1   Class1   1.5   1.5 0.5000  0.5000   3.0   3.0
 Queue2   Class1   3.5   1.0 1.1667  1.1667   3.0   3.0

Fluid Solver:
  St